<a href="https://colab.research.google.com/github/Georgia-MAX-holic/theory/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/GridSearch_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

K 폴드 더 간단하게 

  K-Fold 클래스를 이용한 교차 검증 방법 

  1. 폴드 세트 설정 

  2. For 루프에서 반복적으로 학습/검증 

  3. 폴드 세트 별로 예{측을 평균하여 최종 성능 판별 



  => Cross_val_score()
  함수로 폴드 세트 추출 
  학습 / 예측 평가를 한번에 

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score  
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state= 156)

data = iris_data.data 
label = iris_data.target 

scores = cross_val_score(dt_clf , data ,label , scoring="accuracy" , cv=3)

print("교차 검증별 정확도:",  np.round(scores, 4))
print("평균 검증 정확도 :", np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도 : 0.9667


# **GridSearch - 교차 검증과 최적 하이퍼 파라미터 튜닝 **

사이킷런은 GridSearch CV를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이퍼 파라미터를 도출할 수 있게 함 



In [ ]:
Grid_parameter ={'max_depth':[1,2,3],
                 'min_samples_split':[2,3]}

CV 세트가 3이라면 

파라미터 순화 적용 횟수 6 

     X

CV 세트 수  3 

학습/검증 총 수행 횟수 = 18 

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score


In [7]:
#데이터 로딩 후 학습 data 와 테스트 data 분리 

iris = load_iris()
X_train, X_test , y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                     test_size=0.2 , random_state = 121)

dtree = DecisionTreeClassifier()

## parameter 들을 dic형태로 
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2,3]}

In [12]:
import pandas as pd
#param_grid의 하이퍼 파라미터들을 3개의 train, test set fold로 나누어서 테스트 수행 설정 
# refit=True가 default, True면 가장 좋은 파라미터 설정으로 재 학습 시킴 

grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 
grid_dtree.fit(X_train, y_train)

#GridSearchCV 결과는 cv_results_ 라는 딕셔너리로 저장됨 이를 DataFrame으로 변환 
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[["params", "mean_test_score", "rank_test_score", "split0_test_score", "split1_test_score", "split2_test_score"]]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [13]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00075436, 0.00071184, 0.00050052, 0.00045514, 0.00046412,
        0.00047175]),
 'std_fit_time': array([5.88795748e-05, 2.59803792e-04, 4.49553740e-05, 1.01171167e-05,
        9.06060311e-06, 2.28337572e-05]),
 'mean_score_time': array([0.00043694, 0.0003531 , 0.00025034, 0.00025018, 0.00026822,
        0.00025423]),
 'std_score_time': array([2.86514815e-05, 6.06655438e-05, 1.22980238e-05, 9.52945537e-06,
        3.56306177e-05, 1.10881017e-05]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_sa

In [14]:
print("GridSearchCV 최적 파라미터:", grid_dtree.best_params_)
print("GridSearchCV 최고 정확도: {0:.4f}".format(grid_dtree.best_score_))

# refit=True로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 Estimator를 내포하고 있으므로 predict

pred = grid_dtree.predict(X_test)
print("테스트 데이터 세트 정확도:{0:.4f}".format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750
테스트 데이터 세트 정확도:0.9667


In [16]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환 
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨 
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667
